## Assetwise sign-in

In order to use chrome from your scripting, you should download the relevant [chrome driver](https://googlechromelabs.github.io/chrome-for-testing/#stable) (chromedriver, not chrome!) for your current version of google chrome, then add the path to the driver to your script below.

In [6]:
# User Configuration
user_email = "dane.parks@dot.ohio.gov"
path_to_excel = r"C:\Users\dparks1\Desktop\assetwise_script_demo\reportSummaryData.xlsx"
file_save_path = r"C:\Users\dparks1\Desktop\assetwise_script_demo"

In [7]:
import os
import time
import shutil
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [8]:
# Launch the development/automation browser
options = webdriver.ChromeOptions()
options.add_experimental_option(
    "prefs",
    {
        "download.default_directory": file_save_path,  # Change default directory for downloads
        "download.prompt_for_download": False,  # To auto download the file
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True,  # It will not show PDF directly in chrome
    },
)

driver = webdriver.Chrome(executable_path=r"C:\Users\dparks1\drivers\chrome\chrome.exe", options=options)

WebDriverException: Message: Service C:\Users\dparks1\drivers\chrome\chrome.exe unexpectedly exited. Status code was: 0


In [ ]:
driver.maximize_window()
driver.get("https://ohiodot-it.bentley.com/login.aspx")
print(f"Page title: {driver.title}")

# Wait until the username box appears, then select it
WebDriverWait(driver, timeout=5).until(
    lambda d: d.find_element(By.XPATH, '//*[@id="ContentPlaceHolder1_cmdOIDCLogin"]')
)

IMS_Login = driver.find_element(By.XPATH, '//*[@id="ContentPlaceHolder1_cmdOIDCLogin"]')
IMS_Login.click()

In [ ]:
# Sign in
username_input = driver.find_element(By.XPATH, '//*[@id="identifierInput"]').send_keys(
    user_email
)
submit_button = driver.find_element(By.XPATH, '//*[@id="sign-in-button"]')
submit_button.click()

In [5]:
# Get the list of SFNs from the excel sheet
import pandas as pd

# Load the excel sheet, 5th tab, Column 'R'=index 17
df = pd.read_excel(path_to_excel, sheet_name=0)

In [6]:
# Store Column in list
sfn_list = df["Asset Code"].tolist()

## Loop through the list of SFNs

In [7]:
def find_first_iframe(driver):
    # Find all iframes on the current page
    iframes = driver.find_elements(By.TAG_NAME, "iframe")

    driver.switch_to.frame(iframes[1])  # Switch to the iframe by index

    try:
        # Try to locate and click the element inside the iframe
        view_pdf_1 = driver.find_element(
            By.XPATH,
            '//*[@id="ctl00_ContentPlaceHolder1_upRASContent"]/table/tbody/tr[3]/td[2]/span',
        )
        view_pdf_1.click()
        print("Element found and clicked!")
        return  # Exit the function after successful interaction
    except Exception as e:
        # Handle case where element is not found in this iframe
        print(f"Element not found in iframe: {e}")
        pass

    # Switch back to the parent frame after checking the current iframe
    driver.switch_to.parent_frame()

In [8]:
def find_second_iframe(driver):
    # Find all iframes on the current page
    iframes = driver.find_elements(By.TAG_NAME, "iframe")
    
    driver.switch_to.frame(iframes[1])  # Switch to the iframe by index
    print(f"Switched to iframe")

    try:
        # Wait for the element inside the iframe
        view_pdf_2 = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="ContentPlaceHolder1_printSections_cmdViewPDFBottom"]')
            )
        )
        view_pdf_2.click()
        print("Element found and clicked in second iframe!")
        return  # Exit after successful interaction
    except Exception as e:
        # //TODO - Figure out which frames these are in and remove the errors
        # print(f"Element not found in iframe {index}: {e}")
        pass

    # Switch back to the parent frame after checking the current iframe
    driver.switch_to.parent_frame()

In [9]:
# Temporarily store an SFN in a variable for testing
for sfn in sfn_list:
    # Hacky, but the loading screen breaks the script...
    time.sleep(10)
    
    # Select the search bar
    search_bar = input_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Search Assets']"))
    )
    search_bar.click()

    search_bar.send_keys(sfn)
    search_bar.send_keys(Keys.RETURN)

    try:
        # Wait until the iframe is present and locate it
        iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "iframe"))  # Locate the iframe by tag name
        )
    
        # Switch to the iframe context
        driver.switch_to.frame(iframe)
    
        # Now, locate the link <a> inside the iframe
        link_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'bridgedetail.aspx')]"))
        )
    
        # Click the link or extract the href
        url = link_element.get_attribute("href")
        print(f"URL found: {url}")
        driver.get(url)
    
        # Optionally: Switch back to the main content
        driver.switch_to.default_content()
    
    except Exception as e:
        print(f"An error occurred: {e}")

    wait = WebDriverWait(driver, timeout=20)
    wait_1 = wait.until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="tabAsset"]/span/span/span'))
        )
    asset_info = driver.find_element(By.XPATH, '//*[@id="tabAsset"]/span/span/span')
    asset_info.click()
    
    wait_2 = wait.until(
        EC.element_to_be_clickable(
            (
                By.XPATH,
                '//*[@id="completedReportsGridDiv"]/div/div[2]/div/div[1]/div[3]/div[1]/div/div[10]/div',
            )
        )
    )
    hamburger = driver.find_element(
        By.XPATH,
        '//*[@id="completedReportsGridDiv"]/div/div[2]/div/div[1]/div[3]/div[1]/div/div[10]/div',
    )
    hamburger.click()
    
    time.sleep(1)
    
    # finds the button in the first iframe
    find_first_iframe(driver)
    
    driver.switch_to.parent_frame()
    
    # finds the button in the second iframe
    find_second_iframe(driver)

    driver.switch_to.parent_frame()
    
    close_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "rwCloseButton"))
    )
    
    close_button.click()

URL found: https://ohiodot-it.bentley.com/bridgedetail.aspx?type=0&as_id=79723
Element found and clicked!
Switched to iframe
Element found and clicked in second iframe!
URL found: https://ohiodot-it.bentley.com/bridgedetail.aspx?type=0&as_id=79724
Element found and clicked!
Switched to iframe
Element found and clicked in second iframe!
URL found: https://ohiodot-it.bentley.com/bridgedetail.aspx?type=0&as_id=79725
Element found and clicked!
Switched to iframe
Element found and clicked in second iframe!
URL found: https://ohiodot-it.bentley.com/bridgedetail.aspx?type=0&as_id=79726
Element found and clicked!
Switched to iframe
Element found and clicked in second iframe!
URL found: https://ohiodot-it.bentley.com/bridgedetail.aspx?type=0&as_id=79727
Element found and clicked!
Switched to iframe
Element found and clicked in second iframe!
URL found: https://ohiodot-it.bentley.com/bridgedetail.aspx?type=0&as_id=79728
Element found and clicked!
Switched to iframe
Element found and clicked in s

In [10]:
def rename_files():
    for index, sfn in enumerate(sfn_list):
        if index == 0:
            # Renames the file
            try:
                shutil.copy(
                    Path(file_save_path) / "report.pdf",
                    Path(file_save_path) / f"{sfn}_inspection_report.pdf",
                )
                os.remove(Path(file_save_path) / "report.pdf")
                time.sleep(5)
            # If source and destination are same
            except shutil.SameFileError:
                print("Source and destination represents the same file.")

            # If there is any permission issue
            except PermissionError:
                print("Permission denied.")

            # For other errors
            except:
                print("Error occurred while copying file.")
        else:
            # Renames the file
            try:
                shutil.copy(
                    Path(file_save_path) / f"report ({index}).pdf",
                    Path(file_save_path) / f"{sfn}_inspection_report.pdf",
                )
                os.remove(Path(file_save_path) / f"report ({index}).pdf")
                time.sleep(5)
            # If source and destination are same
            except shutil.SameFileError:
                print("Source and destination represents the same file.")

            # If there is any permission issue
            except PermissionError:
                print("Permission denied.")

            # For other errors
            except Exception as e:
                print(f"Error occurred while copying file. {e}")

In [11]:
rename_files()

Error occurred while copying file. [Errno 2] No such file or directory: 'C:\\Users\\dparks1\\Desktop\\assetwise_script_demo\\report (63).pdf'


# Old Cells